<a href="https://colab.research.google.com/github/jtwya/AOP/blob/main/EX05_03_Reply_Multi_type_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

In [ ]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK_AUTHTOCKEN'))
    ngrok.connect(5000)
    run_with_ngrok(app)


from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage,StickerMessage,ImageMessage,VideoMessage,LocationMessage
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('Line_Channel_access_token'))
handler = WebhookHandler(userdata.get('Line_Channel_secret'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

import google.generativeai as genai
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
def ask_gemini(question):
  response = model.generate_content(question)
  return response.text



@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == "sticker":
            #回覆貼圖
          reply = StickerMessage(
            package_id='446',
            sticker_id='1989'
          )

            #回覆圖片
        elif action == "image":
          reply = ImageMessage(
            original_content_url='https://miro.medium.com/v2/resize:fit:1400/1*uNCVd_VqFOcdxhsL71cT5Q.jpeg',
            preview_image_url='https://miro.medium.com/v2/resize:fit:1400/1*uNCVd_VqFOcdxhsL71cT5Q.jpeg'
          )
        elif action == "video":
          reply = VideoMessage(
              original_content_url='https://www.youtube.com/watch?v=IrPb7cO8BJw',
              preview_image_url='https://media.istockphoto.com/id/500593190/photo/composition-finger-frame-mans-hands-capture-the-sunset.jpg?s=612x612&w=0&k=20&c=S7cuvvC_hlu39Fj5jon6__3DD0j265aAsqvYX4C0lEM='
          )
        elif action == "location":
          reply = LocationMessage(
              title="鴨肉珍 (總店)",
              address="803高雄市鹽埕區五福四路258號",
              latitude=22.624939852640065,
              longitude=120.2811684675895
          )

        else:
          response = ask_gemini(event.message.text)
          reply = TextMessage(text=response)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                    reply
                ]
            )
        )



ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://c21a-35-231-114-239.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [06/May/2025 09:08:31] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 09:08:32] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 09:08:38] "GET /callback HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 09:08:53] "POST /callback HTTP/1.1" 200 -
ERROR:__main__:Exception on /callback [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 902, in dispat